In [17]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import shutil

In [12]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
dataset_dir = '/Users/ajay/Downloads/Grapevine_Leaves_Image_Dataset'  # Path to your dataset folder
base_dir = 'grapevine_split'  # Output folder for split data

# Create train and test directories
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    os.makedirs(train_dir)
    os.makedirs(test_dir)

# Split each class into train/test
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_dir):
        # Create class subdirectories in train and test
        train_class_dir = os.path.join(train_dir, class_name)
        test_class_dir = os.path.join(test_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)

        # Get all image files in the class folder
        images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        
        # Split the images
        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

        # Copy images to train and test folders
        for img in train_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(train_class_dir, img))
        for img in test_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(test_class_dir, img))


In [13]:
print("Classes in training set:", os.listdir(train_dir))
print("Classes in testing set:", os.listdir(test_dir))


Classes in training set: ['Dimnit', 'Ak', 'Ala_Idris', 'Nazli', 'Buzgulu']
Classes in testing set: ['Dimnit', 'Ak', 'Ala_Idris', 'Nazli', 'Buzgulu']


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data generators
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load train data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 400 images belonging to 5 classes.
Found 100 images belonging to 5 classes.


In [15]:
def create_vgg_model(base_model):
    base_model.trainable = False  # Freeze the base model
    x = Flatten()(base_model.output)  # Flatten the output of the base model
    x = Dense(512, activation='relu')(x)  # Fully connected layer
    x = Dropout(0.5)(x)  # Add dropout for regularization
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer
    model = Model(inputs=base_model.input, outputs=predictions)  # Create the model

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.0001),  # Use Adam optimizer
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [21]:
# Initialize and train VGG-16 model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16_model = create_vgg_model(vgg16_base)

print("Training VGG-16 model...")
vgg16_history = vgg16_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20, 
    verbose=1
)

Training VGG-16 model...
Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 58s 5s/step - accuracy: 0.2114 - loss: 2.0369 - val_accuracy: 0.4600 - val_loss: 1.3074
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 71s 5s/step - accuracy: 0.4089 - loss: 1.4002 - val_accuracy: 0.6000 - val_loss: 1.1027
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 60s 5s/step - accuracy: 0.6527 - loss: 0.9907 - val_accuracy: 0.7000 - val_loss: 0.8903
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 62s 5s/step - accuracy: 0.7576 - loss: 0.7496 - val_accuracy: 0.7400 - val_loss: 0.7635
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 67s 5s/step - accuracy: 0.7475 - loss: 0.6720 - val_accuracy: 0.8100 - val_loss: 0.6156
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 70s 5s/step - accuracy: 0.8472 - loss: 0.5067 - val_accuracy: 0.8500 - val_loss: 0.5586
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 68s 5s/step - accuracy: 0.8683 - loss: 0.4662 - val_accuracy: 0.8500 - val_loss: 0.5225
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 69s 5s/step - accuracy: 0.8656 - loss: 0.4280 - val_acc

In [22]:
# Initialize and train VGG-19 model
vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg19_model = create_vgg_model(vgg19_base)

print("Training VGG-19 model...")
vgg19_history = vgg19_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20, 
    verbose=1
)


Training VGG-19 model...
Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 93s 7s/step - accuracy: 0.2283 - loss: 2.1903 - val_accuracy: 0.4400 - val_loss: 1.5709
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 90s 7s/step - accuracy: 0.3739 - loss: 1.4678 - val_accuracy: 0.6600 - val_loss: 1.0736
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 94s 7s/step - accuracy: 0.5448 - loss: 1.1139 - val_accuracy: 0.6900 - val_loss: 0.9317
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 96s 8s/step - accuracy: 0.6624 - loss: 0.8823 - val_accuracy: 0.7600 - val_loss: 0.7725
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 94s 7s/step - accuracy: 0.7518 - loss: 0.7124 - val_accuracy: 0.7800 - val_loss: 0.7046
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 88s 7s/step - accuracy: 0.8432 - loss: 0.6090 - val_accuracy: 0.8200 - val_loss: 0.6467
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 94s 7s/step - accuracy: 0.8417 - loss: 0.5691 - val_accuracy: 0.7800 - val_loss: 0.6234
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 94s 7s/step - accuracy: 0.8786 - loss: 0.5228 - val_acc

In [23]:
# Evaluate both models
print("Evaluating VGG-16 model...")
vgg16_eval = vgg16_model.evaluate(test_generator, verbose=0)
print(f"VGG-16: Accuracy = {vgg16_eval[1]*100:.2f}%, Loss = {vgg16_eval[0]:.4f}")

print("Evaluating VGG-19 model...")
vgg19_eval = vgg19_model.evaluate(test_generator, verbose=0)
print(f"VGG-19: Accuracy = {vgg19_eval[1]*100:.2f}%, Loss = {vgg19_eval[0]:.4f}")

# Save the trained models
vgg16_model.save('vgg16_grapevine_model.keras')
vgg19_model.save('vgg19_grapevine_model.keras')

print("Models saved successfully.")

Evaluating VGG-16 model...
VGG-16: Accuracy = 85.00%, Loss = 0.3849
Evaluating VGG-19 model...
VGG-19: Accuracy = 84.00%, Loss = 0.3930
Models saved successfully.
